In [1]:
pwd

'/home/wsuser/work'

In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)


In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='t5LKWdXqyifGD4sjG534nnPEYmKNuteRdOTlsF7eLKCL',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageprocessing-donotdelete-pr-i5ko2osgklphx7'
object_key = 'Dataset-20221116T041034Z-001.zip'

streaming_body_10 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
if not hasattr(streaming_body_10,"__iter___"):streaming_body_10.__iter__=types.MethodType(__iter__,streaming_body_10)

In [6]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_10.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
import numpy as np
import tensorflow as tf0
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense, MaxPooling2D, Conv2D

In [8]:
pwd

'/home/wsuser/work'

In [9]:
import os
filenames=os.listdir('/home/wsuser/work/Dataset/train')

In [10]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/train', target_size=(64, 64),
                                            batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Dataset/test', target_size=(64, 64), batch_size=5,
                                          color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [11]:
x_train.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}

In [12]:
model = Sequential()

In [13]:
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))

In [14]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [15]:
model.add(Conv2D(32, (3, 3), activation='relu'))

In [16]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [17]:
model.add(Flatten())

In [18]:
model.add(Dense(units=128, activation='relu'))

In [19]:
model.add(Dense(units=6, activation='softmax'))

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit_generator(x_train,steps_per_epoch=47,epochs=10,validation_data=x_test,validation_steps=20)

/tmp/wsuser/ipykernel_164/2219168709.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=47,epochs=10,validation_data=x_test,validation_steps=20)


Epoch 1/10
47/47 [==============================] - 4s 71ms/step - loss: 1.7624 - accuracy: 0.2340 - val_loss: 1.5338 - val_accuracy: 0.3333
Epoch 2/10
47/47 [==============================] - 2s 36ms/step - loss: 1.3581 - accuracy: 0.4766
Epoch 3/10
47/47 [==============================] - 2s 37ms/step - loss: 0.9527 - accuracy: 0.6624
Epoch 4/10
47/47 [==============================] - 2s 40ms/step - loss: 0.7769 - accuracy: 0.6880
Epoch 5/10
47/47 [==============================] - 2s 38ms/step - loss: 0.6254 - accuracy: 0.7863
Epoch 6/10
47/47 [==============================] - 2s 41ms/step - loss: 0.5662 - accuracy: 0.7906
Epoch 7/10
47/47 [==============================] - 2s 41ms/step - loss: 0.4917 - accuracy: 0.8128
Epoch 8/10
47/47 [==============================] - 2s 35ms/step - loss: 0.3959 - accuracy: 0.8511
Epoch 9/10
47/47 [==============================] - 2s 40ms/step - loss: 0.3601 - accuracy: 0.8632
Epoch 10/10
47/47 [==============================] - 2s 39ms/step -

steps_perepoch=total no of images in trainset/batch size=594/6=99
validation_steps=total no of images in testset/batch size=30/6=5

In [22]:
model.save("gesture.h5")

In [23]:
!tar -zcvf image-classification-model_new.tgz gesture.h5

gesture.h5


In [24]:
ls -1

Dataset/
gesture.h5
image-classification-model_new.tgz


In [25]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 9.6 MB/s eta 0:00:01


In [26]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
                  "url":"https://us-south.ml.cloud.ibm.com",
                  "apikey":"xDw74L6yZ6-00_V5LPmBdtv26_xeKcJhGIycBtD606ud"
                }
client=APIClient(wml_credentials)

In [27]:
client=APIClient(wml_credentials)

In [28]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources']if item['entity']["name"]==space_name)['metadata']['id'])

In [30]:
space_uid=guid_from_space_name(client,'gesture')
print("Space UID = " + space_uid)

Space UID = ec93cc07-c340-4e56-8aec-1ff576a3c1e2


In [31]:
client.set.default_space(space_uid)

'SUCCESS'

In [32]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [33]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [35]:
model_details = client.repository.store_model(model="image-classification-model_new.tgz",meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [36]:
model_id

'9c4bd7e8-3209-4085-8cdf-1e2c816a854c'

In [37]:
client.repository.download(model_id,'my_model2.tar.gz')

Successfully saved model content to file: 'my_model2.tar.gz'


'/home/wsuser/work/my_model2.tar.gz'

In [38]:
from keras.models import load_model
from keras.preprocessing import image

In [39]:
model=load_model("gesture.h5")

In [40]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='t5LKWdXqyifGD4sjG534nnPEYmKNuteRdOTlsF7eLKCL',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageprocessing-donotdelete-pr-i5ko2osgklphx7'
object_key = '0.jpg'

streaming_body_11 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
def cgsReadImage(client,bucket,file):
    isr=client.get_object(Bucket=bucket,key=file)
    jpg=isx['Body']
    print(type(jpg))
    if not hasattr(streaming_body_11, "__iter__"): streaming_body_11.__iter__ = types.MethodType( __iter__, streaming_body_11)
    img=skyio.inread(jpg)
    img=image.load_img(streaming_body_11, target_size = (64, 64),grayscale=True)

In [41]:
from keras.preprocessing import image
import numpy as np
img=image.load_img(path, target_size=(150, 150))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)


In [43]:
pred_x = np.argmax(model.predict(x),axis=1)


ValueError: in user code:

    File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/engine/input_spec.py", line 263, in a

In [ ]:
pred[0]

In [ ]:
index=['0','1','2','3','4']

In [ ]:
a=index[pred[0]]
print(a)